## Context
The previous dataset didn't provide the sufficient details for the feature.
This web scraper will create a dataset with the results for each party in each parish ('freguesia') in Portugal (inland and autonomous territories) as well as the votes from the Europe and Outside of Europe circles

### Setup

In [1]:
#%pip install pandas
import pandas as pd
import os

In [2]:
# Base url
site = 'https://www.legislativas2024.mai.gov.pt/assets/static'

# Local path
tchild = site +'/territory-children/territory-children-' # children territory codes url
tresult = site +'/territory-results/territory-results-' # voting results url

portugal = {
                'compensation': [False], 
                'name': ['Portugal'], 
                'territoryKey': ['LOCAL-500000'],
                'distrito': False,
                'concelho': False,
                'level': ['pais']
            }

codes = pd.DataFrame(portugal) # creates codes dataframe, and populates with highest-tier level

### Methods

In [3]:
# Recursively drills down to freguesias, then gets results
def get_results(locals, level):
    for i, row in locals.iterrows():
        get_codes(row, level)
#         match level:
#             case 'freguesia':
#                 freguesias_results = freguesias_results.append(get_json(row['territoryKey']))
#             case 'concelho':
#                 get_results(get_json(row['territoryKey']), 'freguesia')
#             case 'distrito':
#                 get_results(get_json(row['territoryKey']), 'conselho')

# Saves children json from url          
def get_children(territoryKey):
    url = tchild + territoryKey + '.json'
    return pd.read_json(url)

# Saves result json from url
def get_result(territoryKey):
    url = tresult + territoryKey + '-AR.json'
    return pd.read_json(url)

# Populates codes dataframe
def get_codes(row, district, county, level):
    return  {
                'compensation':     row['compensation'],
                'name':             row['name'], 
                'territoryKey':     row['territoryKey'],
                'distrito':         district,
                'concelho':         county,
                'level':            level
            }

def makedir(path):
    working_dir = os.getcwd()
    newpath = working_dir + path
    if not os.path.exists(newpath):
        os.makedirs(newpath)

### Scraping

In [4]:
# Drill down to districts
tkey = portugal['territoryKey'][0]
districts = get_children(tkey)

district_codes = pd.DataFrame(get_codes(districts, False, False, 'distrito'))

In [5]:
county_codes = pd.DataFrame()

# Drill down to counties
for row in district_codes.iterrows():
    tkey = row[1]['territoryKey']
    dist = row[1]['name']
    county = False
    
    counties = get_children(tkey)
    codes = pd.DataFrame(get_codes(counties, dist, county, 'concelho'))
    county_codes = pd.concat([county_codes, codes])

county_codes = county_codes.reset_index(drop=True)

##### Run the below cell once to create the dataset folders

In [6]:
# for row in county_codes.iterrows():
#     dist = row[1]['distrito']
#     county = row[1]['name']
#     path = '/datasets/' + dist + '/' + county + '/'
#     makedir(path)

In [12]:
parish_codes = pd.DataFrame()

# Drill down to parishes
for row in county_codes.iterrows():
    tkey = row[1]['territoryKey']
    dist = row[1]['distrito']
    county = row[1]['name']
    
    parishes = get_children(tkey)
    codes = pd.DataFrame(get_codes(parishes, dist, county, 'freguesia'))
    parish_codes = pd.concat([parish_codes, codes])
    
parish_dict =   {
                    'Longueira/Almograve': 'Longueira, Almograve',
                    'Alverca da Beira/Bouça Cova': 'Alverca da Beira, Bouça Cova',
                    'Valbom/Bogalhal': 'Valbom, Bogalhal',
                    'Santa Cruz/Trindade e Sanjurge': 'Santa Cruz, Trindade e Sanjurge',
                    'Vila Cova do Covelo/Mareco': 'Vila Cova do Covelo, Mareco'
                }

parish_codes =  parish_codes.map(parish_dict).reset_index(drop=True)

TypeError: the first argument must be callable

In [ ]:
parish_codes

### Test

In [8]:
# test = parish_codes.iloc[0]
# tkey = test['territoryKey']
# dist = test['distrito']
# county = test['concelho']
# parish = test['name']

# result = pd.DataFrame(get_result(tkey))

# #path = '\\datasets\\' + dist + '\\' + county + '\\'
# path = 'datasets/' + dist + '/' + county + '/'
# path
# result.to_csv(path + parish + '.csv')

In [10]:
freguesias_results = pd.DataFrame()

for row in parish_codes.iterrows():
    tkey = row[1]['territoryKey']
    dist = row[1]['distrito']
    county = row[1]['concelho']
    parish = row[1]['name']
    
    result = pd.DataFrame(get_result(tkey))
    
    path = 'datasets/' + dist + '/' + county + '/'

    result.to_csv(path + parish + '.csv')

OSError: Cannot save file into a non-existent directory: 'datasets\Beja\Odemira\Longueira'